In [1]:
from sage.all import *

# define the parity check amrtix in standart form of the Steane code
# GF(2) is a short-handle for F2 but you can get other finite fields GF(p^n)
H = matrix(GF(2), # this makes the matrix F2
           [[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
            [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
            [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1]],
        sparse=False)

# dimension of the matrix is (nstabs, 2 * n) where n is the number of physical qubits
nstabs, two_n= H.dimensions()

n = two_n // 2


# define the symplectic form
id = matrix.identity(GF(2), n)
w = block_matrix([[matrix(n, n), id], [id, matrix(n, n)]])

# compute the RIGHT kernel of H * w, this gives stuff that commutes with all stabilizers
centralizer = (H * w).right_kernel()

assert dimension(H.row_space()) == nstabs, 'Stabilizers should be independent!'

# compute the number of logicals as log2(dim(centralizer) - nstabs)
k = log(dimension(centralizer) - nstabs, 2)

print('The Steane code encodes', k, 'logical qubits using', n, 'physical qubits')


The Steane code encodes 1 logical qubits using 7 physical qubits


In [2]:
# a minimal SSC example: 2D Bacon-Shor

Lx = 3
Ly = 3
n2 = Lx * Ly
rows = []

for ix in range(Lx):
    for iy in range(Ly):
        # x check on site
        x_check = [0 for _ in range(2 * n2)]
        x_check[ix * Ly + iy] = 1
        x_check[((ix + 1) % Lx) * Ly + iy] = 1

        # add to row list
        rows += [x_check]
# same for the y checks
for ix in range(Lx):
    for iy in range(Ly):
        y_check = [0 for _ in range(2 * n2)]
        y_check[n + ix * Ly + iy] = 1
        y_check[n + ix * Ly + ((iy + 1) % Ly)] = 1

        rows += [y_check]

H2 = matrix(GF(2), rows, sparse=False)

# define the symplectic form
id2 = matrix.identity(GF(2), n2)
w2 = block_matrix([[matrix(n2, n2), id2], [id2, matrix(n2, n2)]])

# for a subsystem code, we cant subtract just the number of stabilizers since we do not have them. 
# howecer, the center of the check group can be computed as the intersection of the centralizer (kernel of H2 * w) the check group itself (the row space of H2)
centralizer2 = (H2 * w2).right_kernel()
center2 = centralizer2.intersection(H2.row_space())

# we indeed get even a rather nice basis (that is a rather nice set of stabilizer generators)
# you can get the basis in matrix form using center2.basis_matrix()
center2

Vector space of degree 18 and dimension 2 over Finite Field of size 2
Basis matrix:
[0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]

In [3]:
# now we con proceed as before

k2 = log(dimension(centralizer2) - dimension(center2), 2)

print(f'The {Lx}x{Ly} Bacon-Shor code encodes {k2} logical qubit(s) using {n2} physical qubits')


The 3x3 Bacon-Shor code encodes 2 logical qubit(s) using 9 physical qubits
